## Step 1. 데이터 수집하기

In [1]:
!git clone https://github.com/songys/Chatbot_data.git

fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


Step 2. 데이터 전처리하기

In [2]:
%pip install pandas

import pandas as pd

# 데이터 로드
data = pd.read_csv('Chatbot_data/ChatbotData.csv')

# 결측치 제거
data = data.dropna()

# 데이터 확인
print(data.head())

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0
[notice] To update, run: c:\Users\ho128\.pyenv\pyenv-win\versions\3.11.0\python.exe -m pip install --upgrade pip


                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


Step 3. SubwordTextEncoder 사용하기

In [3]:
%pip install tensorflow-datasets

import tensorflow_datasets as tfds

# SubwordTextEncoder를 사용하여 토크나이저 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (sentence for sentence in data['Q']), target_vocab_size=2**13)

# 예시 문장 토크나이징
sample_sentence = "안녕하세요"
tokenized_sentence = tokenizer.encode(sample_sentence)
print(tokenized_sentence)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0
[notice] To update, run: c:\Users\ho128\.pyenv\pyenv-win\versions\3.11.0\python.exe -m pip install --upgrade pip


[6053, 67]


Step 4. 모델 구성하기

In [4]:
%pip install rich

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, MultiHeadAttention, LayerNormalization, Dropout

def transformer_model(vocab_size, num_layers, units, d_model, num_heads, dropout):
    inputs = Input(shape=(None,))
    embedding = Embedding(vocab_size, d_model)(inputs)
    x = embedding

    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
        attn_output = Dropout(dropout)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)

        ffn_output = Dense(units, activation='relu')(out1)
        ffn_output = Dense(d_model)(ffn_output)
        ffn_output = Dropout(dropout)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

    outputs = Dense(vocab_size)(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# 모델 생성
model = transformer_model(
    vocab_size=tokenizer.vocab_size,
    num_layers=2,
    units=512,
    d_model=256,
    num_heads=8,
    dropout=0.1
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


[notice] A new release of pip available: 22.3 -> 25.0
[notice] To update, run: c:\Users\ho128\.pyenv\pyenv-win\versions\3.11.0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  2,209,792 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 256) │  2,103,552 │ embedding[0][0],  │
│ (MultiHeadAttentio… │                   │            │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, None, 256) │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, None, 256) │          0 │ embedding[0][0],  │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, None, 256) │        512 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 512) │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 256) │    131,328 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, None, 256) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, None, 256) │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 256) │        512 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 256) │  2,103,552 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, None, 256) │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, None, 256) │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 256) │        512 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None, 512) │    131,584 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, None, 256) │    131,328 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, None, 256) │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, None, 256) │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 9,163,192 (34.95 MB)

 Trainable params: 9,163,192 (34.95 MB)

 Non-trainable params: 0 (0.00 B)

Step 5. 모델 평가하기

In [5]:
def predict(sentence):
    tokenized_sentence = tokenizer.encode(sentence)
    tokenized_sentence = tf.expand_dims(tokenized_sentence, axis=0)

    predictions = model.predict(tokenized_sentence)
    predicted_id = tf.argmax(predictions[0], axis=-1).numpy()

    predicted_sentence = tokenizer.decode(predicted_id)
    return predicted_sentence

# 예측 예시
input_sentence = "안녕하세요"
output_sentence = predict(input_sentence)
print(output_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
~~ 옷
